# Введение

- A/B-тестирование — это анализ двух маркетинговых стратегий для выбора наиболее эффективной, которая сможет конвертировать трафик в продажи (или трафик в другую целевую метрику) максимально эффективно и результативно. Это одна из ключевых концепций, которую должен знать каждый специалист в области Data Science.

# A/B-тестирование

- В A/B-тестировании мы анализируем результаты двух маркетинговых стратегий, чтобы выбрать лучшую для будущих кампаний.

- В этом и заключается суть A/B-тестирования. Ваша цель может быть в увеличении продаж, подписчиков или трафика, но когда мы выбираем оптимальную стратегию на основе результатов предыдущих кампаний — это и есть A/B-тестирование.

- Для реализации A/B-тестирования на Python нам необходим датасет с данными о двух разных маркетинговых стратегиях, направленных на достижение одной цели.

# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "plotly_white"

# Обработка данных

## Чтение данных

In [ ]:
control_data = pd.read_csv("control_group.csv", sep = ";")
test_data = pd.read_csv("test_group.csv", sep = ";")

In [ ]:
control_data.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2.08.2019,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,3.08.2019,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,4.08.2019,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,5.08.2019,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_data.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768


## Подготовка данных

- В датасете есть ошибки в названиях столбцов. Перед продолжением зададим новые имена:

In [ ]:
control_data.columns = ["Название кампании", "Дата", "Сумма затрат",
                       "Количество показов", "Охват", "Клики на сайте",
                       "Поисковые запросы", "Просмотры контента", "Добавлено в корзину",
                       "Покупки"]

test_data.columns = ["Название кампании", "Дата", "Сумма затрат",
                       "Количество показов", "Охват", "Клики на сайте",
                       "Поисковые запросы", "Просмотры контента", "Добавлено в корзину",
                       "Покупки"]

## Проверка данных

- Проверим, есть ли в данных пропущенные значения:

In [ ]:
control_data.isnull().sum()

,0
Название кампании,0
Дата,0
Сумма затрат,0
Количество показов,1
Охват,1
Клики на сайте,1
Поисковые запросы,1
Просмотры контента,1
Добавлено в корзину,1
Покупки,1


In [ ]:
test_data.isnull().sum()

,0
Название кампании,0
Дата,0
Сумма затрат,0
Количество показов,0
Охват,0
Клики на сайте,0
Поисковые запросы,0
Просмотры контента,0
Добавлено в корзину,0
Покупки,0


## Пропущенные значения

- В датасете контрольной кампании есть пропущенные значения в одной строке. Заполним их средним значением каждого столбца:

In [ ]:
control_data["Количество показов"].fillna(value=control_data["Количество показов"].mean(), inplace=True)
control_data["Охват"].fillna(value=control_data["Охват"].mean(), inplace=True)
control_data["Клики на сайте"].fillna(value=control_data["Клики на сайте"].mean(), inplace=True)
control_data["Поисковые запросы"].fillna(value=control_data["Поисковые запросы"].mean(), inplace=True)
control_data["Просмотры контента"].fillna(value=control_data["Просмотры контента"].mean(), inplace=True)
control_data["Добавлено в корзину"].fillna(value=control_data["Добавлено в корзину"].mean(), inplace=True)
control_data["Покупки"].fillna(value=control_data["Покупки"].mean(), inplace=True)

<ipython-input-10-82be8ba43b4c>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  control_data["Количество показов"].fillna(value=control_data["Количество показов"].mean(), inplace=True)
<ipython-input-10-82be8ba43b4c>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

## Объединение наборов данных

- Создадим новый датасет, объединив оба набора данных:

In [ ]:
ab_data = control_data.merge(test_data, how="outer").sort_values(["Дата"])
ab_data = ab_data.reset_index(drop=True)
print(ab_data.head())

  Название кампании        Дата  Сумма затрат  Количество показов    Охват  \
0  Control Campaign   1.08.2019          2280             82702.0  56930.0   
1     Test Campaign   1.08.2019          3008             39550.0  35820.0   
2  Control Campaign  10.08.2019          2149            117624.0  91257.0   
3     Test Campaign  10.08.2019          2790             95054.0  79632.0   
4  Control Campaign  11.08.2019          2490            115247.0  95843.0   

   Клики на сайте  Поисковые запросы  Просмотры контента  Добавлено в корзину  \
0          7016.0             2290.0              2159.0               1819.0   
1          3038.0             1946.0              1069.0                894.0   
2          2277.0             2475.0              1984.0               1629.0   
3          8125.0             2312.0              1804.0                424.0   
4          8137.0             2941.0              2486.0               1887.0   

   Покупки  
0    618.0  
1    255.0  
2    

<ipython-input-32-805e7048e392>:1: UserWarning:

You are merging on int and float columns where the float values are not equal to their int representation.



## Проверка данных на нормальность распределения

In [ ]:
stat_control, p_value_control = stats.shapiro(control_data['Покупки'])
stat_test, p_value_test = stats.shapiro(test_data['Охват'])
print(f'Для контрольной кампании p-value = {p_value_control}')
print(f'Для тестовой кампании p-value = {p_value_test}')

Для контрольной кампании p-value = 0.11144612212042365
Для тестовой кампании p-value = 0.05699605107347176


## Название кампании

- Прежде чем продолжить, проверим, содержит ли датасет одинаковое количество наблюдений для обеих кампаний:

In [ ]:
ab_data["Название кампании"].value_counts()

,count
Название кампании,
Control Campaign,30
Test Campaign,30


### Датасет содержит по 30 наблюдений для каждой кампании. Теперь приступим к A/B-тестированию, чтобы определить наилучшую маркетинговую стратегию.

# Лучшая маркетинговая стратегия

- A/B-тестирование для определения лучшей маркетинговой стратегии
    - Для начала A/B-тестирования я проанализирую взаимосвязь между количеством показов из обеих кампаний и затраченными на них средствами:

In [ ]:
figure = px.scatter(data_frame=ab_data,
                    x="Количество показов",
                    y="Сумма затрат",
                    size="Сумма затрат",
                    color="Название кампании",
                    trendline="ols")
figure.update_layout(
    width=1200,
    height=600
)
figure.show()

# Сравнение одинаковых признаков между выборками

## Общее количество поисковых запросов

- Согласно затраченной сумме, контрольная кампания обеспечила больше показов. Теперь проанализируем количество поисковых запросов на сайте для обеих кампаний:

In [ ]:
counts = [sum(control_data["Поисковые запросы"]), sum(test_data["Поисковые запросы"])]
colors = ['#636EFA','#EF553B']

fig = go.Figure()

fig.add_trace(go.Bar(
    x=["Контрольная кампания"],
    y=[counts[0]],
    name="Общие поиски (Контрольная)",
    marker_color=colors[0],
    marker_line=dict(color='black', width=3))
)

fig.add_trace(go.Bar(
    x=["Тестовая кампания"],
    y=[counts[1]],
    name="Общие поиски (Тестовая)",
    marker_color=colors[1],
    marker_line=dict(color='black', width=3))
)

fig.update_layout(
    title_text='Контрольная vs Тестовая: Поисковые запросы',
    xaxis_title="Кампания",
    yaxis_title="Количество запросов",
    width=800,
    height=600,
    showlegend=True,
    legend=dict(
        title="Тип запросов",
        x=1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    uniformtext_minsize=12
)

fig.update_traces(
    texttemplate='%{y:,}',
    textposition='outside',
    textfont_size=16,
    textangle=0
)

fig.show()

In [ ]:
# ### Статистический анализ
def calculate_statistical_significance(control, test, metric_name):
    """Расчет t-теста и размера эффекта"""
    t_stat, p_value = stats.ttest_ind(control, test, equal_var=False)
    pooled_std = np.sqrt((np.var(control) + np.var(test)) / 2)
    cohens_d = (np.mean(control) - np.mean(test)) / pooled_std

    print(f"\nМетрика: {metric_name}")
    print(f"p-value: {p_value:.4f}")
    print(f"Эффект Коэна: {cohens_d:.2f}")

# Проверка значимости
calculate_statistical_significance(
    control_data['Поисковые запросы'],
    test_data['Поисковые запросы'],
    'Поисковые запросы'
)


Метрика: Поисковые запросы
p-value: 0.2540
Эффект Коэна: -0.30


## Клики на сайте

- Тестовая кампания привела к большему количеству поисков на сайте. Теперь проанализируем количество кликов на сайте для обеих кампаний:

In [ ]:
# Данные для графика
counts = [sum(control_data["Клики на сайте"]), sum(test_data["Клики на сайте"])]
colors = ['#636EFA','#EF553B']

fig = go.Figure()

# Добавление столбцов для каждой кампании
fig.add_trace(go.Bar(
    x=["Контрольная кампания"],
    y=[counts[0]],
    name="Клики (Контрольная)",
    marker_color=colors[0],
    marker_line=dict(color='black', width=3)
))

fig.add_trace(go.Bar(
    x=["Тестовая кампания"],
    y=[counts[1]],
    name="Клики (Тестовая)",
    marker_color=colors[1],
    marker_line=dict(color='black', width=3)
))

# Настройка макета
fig.update_layout(
    title_text='Контрольная vs Тестовая: Клики на сайте',
    xaxis_title="Кампания",
    yaxis_title="Количество кликов",
    width=800,
    height=600,
    showlegend=True,
    legend=dict(
        title="Тип кликов",
        x=1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    uniformtext_minsize=12
)

# Настройка отображения значений
fig.update_traces(
    texttemplate='%{y:,}',
    textposition='outside',
    textfont_size=16,
    textangle=0
)

fig.show()

In [ ]:
# Проверка значимости
calculate_statistical_significance(
    control_data['Клики на сайте'],
    test_data['Клики на сайте'],
    'Клики на сайте'
)


Метрика: Клики на сайте
p-value: 0.1141
Эффект Коэна: -0.42


## Просмотры контента

- Тестовая кампания лидирует по количеству кликов на сайте. Теперь проанализируем, какой объём контента был просмотрен после перехода на сайт в обеих кампаниях:

In [ ]:
# Данные для графика
counts = [sum(control_data["Просмотры контента"]), sum(test_data["Просмотры контента"])]
colors = ['#636EFA','#EF553B']

fig = go.Figure()

# Добавление столбцов для каждой кампании
fig.add_trace(go.Bar(
    x=["Контрольная кампания"],
    y=[counts[0]],
    name="Просмотры (Контрольная)",
    marker_color=colors[0],
    marker_line=dict(color='black', width=3)
))

fig.add_trace(go.Bar(
    x=["Тестовая кампания"],
    y=[counts[1]],
    name="Просмотры (Тестовая)",
    marker_color=colors[1],
    marker_line=dict(color='black', width=3)
))

# Настройка макета
fig.update_layout(
    title_text='Контрольная vs Тестовая: Просмотры контента',
    xaxis_title="Кампания",
    yaxis_title="Количество просмотров",
    width=800,
    height=600,
    showlegend=True,
    legend=dict(
        title="Тип просмотров",
        x=1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    uniformtext_minsize=12
)

# Настройка отображения значений
fig.update_traces(
    texttemplate='%{y:,}',
    textposition='outside',
    textfont_size=16,
    textangle=0
)

fig.show()

In [ ]:
# Проверка значимости
calculate_statistical_significance(
    control_data['Просмотры контента'],
    test_data['Просмотры контента'],
    'Просмотры контента'
)


Метрика: Просмотры контента
p-value: 0.6300
Эффект Коэна: 0.13


### Аудитория контрольной кампании просмотрела больше контента, чем тестовой. Хотя разница невелика, учитывая, что кликов на сайте в контрольной кампании было меньше, её вовлеченность на сайте выше, чем у тестовой кампании.

## Товары, добавленные в корзину

- Проанализируем количество товаров, добавленных в корзину, для обеих кампаний:

In [ ]:
# Данные для графика
counts = [sum(control_data["Добавлено в корзину"]),
          sum(test_data["Добавлено в корзину"])]
colors = ['#636EFA','#EF553B']

fig = go.Figure()

# Добавление столбцов для каждой кампании
fig.add_trace(go.Bar(
    x=["Контрольная кампания"],
    y=[counts[0]],
    name="Добавлено в корзину (Контрольная)",
    marker_color=colors[0],
    marker_line=dict(color='black', width=3)
))

fig.add_trace(go.Bar(
    x=["Тестовая кампания"],
    y=[counts[1]],
    name="Добавлено в корзину (Тестовая)",
    marker_color=colors[1],
    marker_line=dict(color='black', width=3))
)

# Настройка макета
fig.update_layout(
    title_text='Контрольная vs Тестовая: Добавлено в корзину',
    xaxis_title="Кампания",
    yaxis_title="Количество добавлений",
    width=950,
    height=600,
    showlegend=True,
    legend=dict(
        title="Тип добавлений",
        x=1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    uniformtext_minsize=12
)

# Настройка отображения значений
fig.update_traces(
    texttemplate='%{y:,}',
    textposition='outside',
    textfont_size=16,
    textangle=0
)

fig.show()

In [ ]:
# Проверка значимости
calculate_statistical_significance(
    control_data['Добавлено в корзину'],
    test_data['Добавлено в корзину'],
    'Добавлено в корзину'
)


Метрика: Добавлено в корзину
p-value: 0.0001
Эффект Коэна: 1.14


### Несмотря на низкое количество кликов на сайте, в контрольной кампании в корзину было добавлено больше товаров.

## Затраченные средства

- Проанализируем затраченные средства на обеих кампаниях:

In [ ]:
# Данные для графика
counts = [sum(control_data["Сумма затрат"]), sum(test_data["Сумма затрат"])]
colors = ['#636EFA','#EF553B']

fig = go.Figure()

# Добавление столбцов для каждой кампании
fig.add_trace(go.Bar(
    x=["Контрольная кампания"],
    y=[counts[0]],
    name="Сумма затрат (Контрольная)",
    marker_color=colors[0],
    marker_line=dict(color='black', width=3)
))

fig.add_trace(go.Bar(
    x=["Тестовая кампания"],
    y=[counts[1]],
    name="Сумма затрат (Тестовая)",
    marker_color=colors[1],
    marker_line=dict(color='black', width=3)
))

# Настройка макета
fig.update_layout(
    title_text='Контрольная vs Тестовая: Сумма затрат',
    xaxis_title="Кампания",
    yaxis_title="Сумма затрат",
    width=800,
    height=600,
    showlegend=True,
    legend=dict(
        title="Тип затрат",
        x=1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    uniformtext_minsize=12
)

# Настройка отображения значений
fig.update_traces(
    texttemplate='%{y:,.0f}',
    textposition='outside',
    textfont_size=16,
    textangle=0
)

fig.show()

In [ ]:
# Проверка значимости
calculate_statistical_significance(
    control_data['Сумма затрат'],
    test_data['Сумма затрат'],
    'Сумма затрат'
)


Метрика: Сумма затрат
p-value: 0.0043
Эффект Коэна: -0.78


### Затраты на тестовую кампанию выше, чем на контрольную. Однако, как видно из данных, контрольная кампания обеспечила больше просмотров контента и товаров в корзине, что делает её более эффективной по сравнению с тестовой.

## Совершённые покупки

- Проанализируем покупки, совершённые в обеих кампаниях:

In [ ]:
# Данные для графика
counts = [sum(control_data["Покупки"]), sum(test_data["Покупки"])]
colors = ['#636EFA','#EF553B']

fig = go.Figure()

# Добавление столбцов для каждой кампании
fig.add_trace(go.Bar(
    x=["Контрольная кампания"],
    y=[counts[0]],
    name="Покупки (Контрольная)",
    marker_color=colors[0],
    marker_line=dict(color='black', width=3)
))

fig.add_trace(go.Bar(
    x=["Тестовая кампания"],
    y=[counts[1]],
    name="Покупки (Тестовая)",
    marker_color=colors[1],
    marker_line=dict(color='black', width=3)
))

# Настройка макета
fig.update_layout(
    title_text='Контрольная vs Тестовая: Покупки',
    xaxis_title="Кампания",
    yaxis_title="Количество покупок",
    width=800,
    height=600,
    showlegend=True,
    legend=dict(
        title="Тип покупок",
        x=1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    uniformtext_minsize=12
)

# Настройка отображения значений
fig.update_traces(
    texttemplate='%{y:,}',
    textposition='outside',
    textfont_size=16,
    textangle=0
)

fig.show()

In [ ]:
# Проверка значимости
calculate_statistical_significance(
    control_data['Покупки'],
    test_data['Покупки'],
    'Покупки'
)


Метрика: Покупки
p-value: 0.9756
Эффект Коэна: 0.01


### Разница в покупках между рекламными кампаниями составляет около 1%. Поскольку контрольная кампания привела к большему количеству продаж при меньшем бюджете на маркетинг — контрольная кампания одерживает победу!

# Анализ метрик для определения конверсии

## Название кампании

- Проанализируем метрики, чтобы определить, какая рекламная кампания обеспечивает более высокую конверсию. Сначала я изучу взаимосвязь между количеством кликов на сайте и просмотрами контента в обеих кампаниях:

In [ ]:
figure = px.scatter(data_frame=ab_data,
                    x="Просмотры контента",
                    y="Клики на сайте",
                    size="Клики на сайте",
                    color="Название кампании",
                    trendline="ols")
figure.update_layout(
    width=1200,
    height=600
)
figure.show()

### Количество кликов на сайте выше в тестовой кампании, но вовлеченность (конверсия) с этих кликов выше в контрольной кампании. Победитель — контрольная кампания!

## Добавление в корзину

- Я проанализирую взаимосвязь между объёмом просмотренного контента и количеством товаров, добавленных в корзину, для обеих кампаний:

In [ ]:
figure = px.scatter(data_frame=ab_data,
                    x="Добавлено в корзину",
                    y="Просмотры контента",
                    size="Добавлено в корзину",
                    color="Название кампании",
                    trendline="ols")
figure.update_layout(
    width=1200,
    height=600
)
figure.show()

### Контрольная кампания побеждает!

## Покупки

- Проанализируем взаимосвязь между количеством товаров, добавленных в корзину, и объёмом продаж для обеих кампаний:

In [ ]:
figure = px.scatter(data_frame=ab_data,
                    x="Покупки",
                    y="Добавлено в корзину",
                    size="Покупки",
                    color="Название кампании",
                    trendline="ols")
figure.update_layout(
    width=1200,
    height=600
)
figure.show()

### Хотя контрольная кампания привела к большему количеству продаж и товаров в корзине, конверсия тестовой кампании выше.

# Воронка конверсий по кампаниям

In [ ]:
def plot_conversion_funnel(data):
    """Визуализация воронки конверсии"""
    funnel_steps = [
        "Количество показов", "Клики на сайте",
        "Добавлено в корзину", "Покупки"
    ]

    fig = make_subplots(rows=1, cols=2,
                        specs=[[{"type": "funnel"}, {"type": "funnel"}]],
                        subplot_titles=["Control Campaign", "Test Campaign"])

    colors = {"Control Campaign": "#636EFA", "Test Campaign": "#EF553B"}

    for i, campaign in enumerate(data["Название кампании"].unique()):
        campaign_data = data[data["Название кампании"] == campaign]
        values = [campaign_data[step].sum() for step in funnel_steps]

        fig.add_trace(
            go.Funnel(
                name=campaign,
                y=funnel_steps,
                x=values,
                textinfo="value+percent initial",
                marker={"color": colors[campaign]},
                textfont={"size": 14}
            ),
            row=1, col=i+1
        )

    fig.update_layout(
        title_text="<b>Воронка конверсии по кампаниям</b>",
        showlegend=True,
        funnelgap=0.3,
        width=1400,
        height=600,
        margin=dict(l=100, r=100)
    )
    fig.show()

plot_conversion_funnel(ab_data)

# Заключение

- По результатам проведённых A/B-тестов, контрольная кампания показала:  
  ✅ **Более высокие продажи и вовлечённость**: Посетители чаще просматривали товары, добавляли их в корзину и совершали покупки.  
  ✅ **Широкий охват**: Эффективна для продвижения ассортимента товаров массовой аудитории.  

- Тестовая кампания выделилась:  
  📊 **Высокой конверсией**: Лучшее соотношение добавлений в корзину к покупкам для целевых товаров.  
  🎯 **Точечным воздействием**: Подходит для продвижения специфических продуктов узкой аудитории.  

**Рекомендация**:  
Используйте тестовую кампанию для точечных промо-акций, а контрольную — для масштабных распродаж с широким ассортиментом.